# PHYS 441/DATA SCI 421 FINAL

## Your Project Title

---

By YOUR NAME  
06 December 2023

### Instructions

This template for the final project is written as a Jupyter Notebook, which allows for a nice mix of standard text as well as software and plotting. 

Text in a Jupyter notebook is written using Markdown. The formatting is fairly straightforward, with many tutorials online ([this one is fairly comprehensive](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd)). 

Follow this template when completing the final project.

## Introduction

*Write one to two paragraphs providing context for the project. What measurement/analysis is going to be attempted and why is this important in the field in question.*

## Data

*Introduce the data by describing the measurements, how they were collected, and how you managed to curate them for this project. In many cases it will also be appropriate to show a figure of the data. Label axes and provide units where appropriate, figures require a caption (which can be written in a markdown cell immediately following any plots.*

## Analysis/Results

*This is the bulk of the project - use statistical methods to analyze the data. This will require text to explain what you are doing (e.g., if you define a likelihood for the data, then you should write about how you arrived at that likelihood and write it out). The text should appear in markdown cells. This section will also present the software that executes the analysis, as well as any corresponding plots. Figures should have a caption (in a markdown cell). You will need multiple cells to complete this section. Units and uncertainties are important to report throughout this section (we never discussed significant figures in class, but please adhere to this in your reporting).* 

In [ ]:
# Analysis 1

*Additional text example*

In [ ]:
# Analysis 2

## Conclusions

*Write a few sentences summarizing the big take away from your analysis (e.g., I attempted to fit observations to measure the mass of an electron and I found $m_e = MMM.MMM \pm 0.MMM\,\mathrm{MeV}$...)*